In [1]:
import os
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

23/04/12 20:18:06 WARN Utils: Your hostname, X resolves to a loopback address: 127.0.1.1; using 172.30.79.3 instead (on interface eth0)
23/04/12 20:18:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/12 20:18:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# path = 'q1/data/soc-LiveJournal1Adj.txt'
path = 'q1/data/small.txt'

In [34]:
txt = sc.textFile(path)

### Pipeline sketch:
The problem of finding the number of mutual friends of two people can be converted to counting how many times they co-occur in another person's friend list. Given a user, for each user not in its friend list, we count the occurence of 

In [35]:
def get_friends(row):
    _, friends = row.split('\t')
    if not friends:
        return []
    friends = [int(friend_id) for friend_id in friends.split(',')]
    return friends

In [36]:
def get_real_friends(row):
    user, friends = row.split('\t')
    if not friends:
        return []
    user = int(user)
    return [(user, int(f)) for f in friends.split(',')]

In [37]:
def get_pairs(friends):
    pairs = []
    if len(friends) < 2:
        return pairs
    for i in range(len(friends) - 1):
        for j in range(i + 1, len(friends)):
            pairs.append((friends[i], friends[j]))
            pairs.append((friends[j], friends[i]))
    return pairs

In [48]:
from operator import itemgetter
def rank_friends(friends):
    friends = sorted(friends, key=lambda x: (-x[1], x[0]))
    return [f[0] for f in friends[:10]]

In [39]:
from operator import add
friends = txt.map(get_friends).flatMap(get_pairs)

In [40]:
real_friends = txt.flatMap(get_real_friends)

In [41]:
friends = friends.subtract(real_friends)

In [42]:
counts = friends.map(lambda p: (p, 1)).reduceByKey(add)

In [43]:
counts.collect()

[((521, 317), 39),
 ((946, 800), 28),
 ((627, 699), 26),
 ((291, 823), 41),
 ((251, 875), 27),
 ((819, 55), 37),
 ((214, 920), 42),
 ((566, 232), 34),
 ((527, 623), 40),
 ((98, 132), 27),
 ((806, 604), 40),
 ((657, 317), 36),
 ((623, 991), 38),
 ((991, 455), 37),
 ((458, 68), 47),
 ((930, 948), 39),
 ((498, 948), 41),
 ((51, 251), 39),
 ((255, 455), 37),
 ((460, 98), 29),
 ((617, 741), 30),
 ((335, 279), 30),
 ((586, 476), 44),
 ((248, 586), 37),
 ((568, 386), 40),
 ((451, 551), 36),
 ((552, 46), 37),
 ((602, 516), 39),
 ((922, 516), 44),
 ((42, 260), 36),
 ((416, 46), 30),
 ((449, 737), 35),
 ((282, 716), 38),
 ((628, 642), 33),
 ((399, 143), 46),
 ((143, 923), 39),
 ((859, 547), 40),
 ((182, 596), 27),
 ((432, 762), 29),
 ((150, 540), 35),
 ((436, 362), 40),
 ((989, 801), 43),
 ((575, 291), 49),
 ((575, 519), 38),
 ((327, 71), 44),
 ((703, 583), 38),
 ((204, 182), 36),
 ((226, 692), 37),
 ((280, 362), 50),
 ((595, 583), 42),
 ((152, 462), 36),
 ((362, 96), 35),
 ((248, 26), 33),
 ((7

In [45]:
user2counts = counts.map(lambda triple: (triple[0][0], (triple[0][1], triple[1]))).groupByKey().mapValues(list)

In [49]:
ranked_list = user2counts.map(lambda it: (it[0], rank_friends(it[1])))

In [61]:
df = ranked_list.toDF()

ParseException: 
Syntax error at or near end of input(line 1, pos 4)

== SQL ==
user
----^^^


In [62]:
df.show()

+---+--------------------+
| _1|                  _2|
+---+--------------------+
|460|[851, 717, 882, 3...|
|248|[417, 929, 3, 288...|
|568|[842, 103, 390, 7...|
|552|[508, 738, 332, 1...|
|416|[494, 979, 59, 35...|
|628|[84, 350, 361, 56...|
|432|[212, 153, 428, 3...|
|436|[494, 135, 212, 1...|
|204|[900, 108, 708, 9...|
|280|[653, 431, 836, 3...|
|152|[366, 231, 417, 3...|
|228|[900, 286, 426, 8...|
|600|[917, 15, 424, 85...|
|376|[307, 743, 227, 3...|
|208|[301, 818, 339, 6...|
|792|[575, 587, 600, 2...|
|444|[36, 591, 569, 92...|
|604|[672, 560, 7, 12,...|
|904|[277, 958, 49, 59...|
|776|[214, 339, 637, 8...|
+---+--------------------+
only showing top 20 rows



In [72]:
df.foreach(lambda x: print(str(x['_1']) + '\t' + ','.join([str(f) for f in x['_2']])))

627	673,43,98,215,649,767,81,235,338,863
291	343,849,15,300,101,558,657,942,168,354
251	532,1,871,87,170,247,12,36,45,329
819	532,608,863,61,107,361,34,72,711,437
527	214,533,971,36,91,401,137,146,685,727
623	935,337,186,262,280,352,469,135,168,226
991	206,458,584,426,196,444,521,36,408,442
51	136,997,711,78,191,407,558,230,317,541
255	68,34,439,672,918,490,901,997,199,674
335	164,842,114,302,506,649,929,17,18,140
451	997,28,359,352,365,862,374,726,895,74
399	530,847,874,182,315,107,448,721,979,470
143	123,36,266,914,426,496,659,997,397,685
859	36,319,40,621,227,662,725,819,863,917
575	386,469,287,494,158,332,361,615,726,988
327	997,139,191,672,214,470,895,898,32,95
703	994,178,186,527,569,532,864,91,239,590
595	917,661,78,327,107,226,481,11,191,200
811	483,224,54,819,489,979,484,863,60,150
351	458,355,395,390,140,392,469,598,726,806
55	954,435,444,506,651,809,18,108,254,580
467	547,796,584,710,444,346,698,964,72,85
727	44,264,579,917,239,527,710,964,366,464
751	525,897,851,379,769,109

In [ ]:
ids = [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]

In [74]:
! python q1.py

python: can't open file 'q1.py': [Errno 2] No such file or directory


In [75]:
! ls

hw1.docx  hw1_submission_template.tex  q1	 q1_out      q2
hw1.pdf   hw1_submissiontemplate.docx  q1.ipynb  q1_out.txt  q4
